# Python网络爬虫 - 股票数据爬取 
#### Author: Arianna.H
#### Date: Oct 12, 2019
  
  ___
Task: 获取东方财富网中当日沪深A股股票名称和交易数据 

original url: http://quote.eastmoney.com/center/gridlist.html#hs_a_board   

idea: urllib.request—>response—>bs4 parser—>get and transform data—>save csv file  

problem: 网页改版后，源代码没有table信息，股票信息不能从html网页直接获取，需利用开发者工具分析网页请求来源（通过js加载）  

In [103]:
pd.set_option('display.max_columns', None)

In [63]:
import pandas as pd
import urllib.request as r
import bs4
from bs4 import BeautifulSoup

* 使用Google Chrome，F12进入Devtool，通过record点击下一页的网页请求，发现数据通过js加载，复制js链接：

沪深A股
url_shsz="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80,m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507749"

上证A股
url_sh="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,\f128,f136,f115,f152&_=1570855507751"

深证A股
url_sz="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507753"
   
   
* 观察链接得知以下的规律：  
  
pn为page number, pz为page zise； fs=m:0为深证，fs=m:1为上证   
  
  

In [135]:
#通过观察解析后的沪深A股url，得知total size=3823
#修改pz=20为pz=3823，可在一页中获取所有股票数据

url_shsz='http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672\
&pn=1&pz=3823&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,\
m:0+t:80,m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,\
f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507749'

In [136]:
#模拟浏览器访问反爬取沪深A股
#读取第一页数据

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
request = r.Request(url_shsz, headers=headers)
response = r.urlopen(request)
html = response.read().decode('utf-8','ignore')
soup = BeautifulSoup(html,'html.parser')

In [137]:
print(soup.prettify())

jQuery1124048191705730265766_1570855507672({"rc":0,"rt":6,"svr":183020361,"lt":1,"full":1,"data":{"total":3823,"diff":[{"f1":2,"f2":3.14,"f3":10.18,"f4":0.29,"f5":803959,"f6":246812557.0,"f7":11.58,"f8":5.66,"f9":16.12,"f10":10.07,"f11":0.0,"f12":"002619","f13":0,"f14":"艾格拉斯","f15":3.14,"f16":2.81,"f17":2.85,"f18":2.85,"f20":5792974165,"f21":4461350679,"f22":0.0,"f23":0.88,"f24":-3.09,"f25":-6.82,"f62":58809544.0,"f115":9.89,"f128":"-","f140":"-","f141":"-","f136":"-","f152":2},{"f1":2,"f2":3.71,"f3":10.09,"f4":0.34,"f5":552777,"f6":196760400.0,"f7":13.65,"f8":6.58,"f9":-105.98,"f10":1.97,"f11":0.0,"f12":"002622","f13":0,"f14":"融钰集团","f15":3.71,"f16":3.25,"f17":3.37,"f18":3.37,"f20":3116400000,"f21":3116316525,"f22":0.0,"f23":2.42,"f24":18.15,"f25":21.24,"f62":42570310.0,"f115":153.74,"f128":"-","f140":"-","f141":"-","f136":"-","f152":2},{"f1":2,"f2":6.01,"f3":10.07,"f4":0.55,"f5":174752,"f6":102385024.0,"f7":6.41,"f8":5.62,"f9":24.19,"f10":6.75,"f11":0.0,"f12":"600778","f13":1,"f14":"

In [139]:
#读取json文档，转换为python dictionary格式
jsondata = soup.prettify().split('(')[1].split(')')[0]
dictdata = json.loads(jsondata)
share_info = dictdata['data']['diff']

In [140]:
pd.DataFrame.from_dict(share_info).head()

,f1,f10,f11,f115,f12,f128,f13,f136,f14,f140,f141,f15,f152,f16,f17,f18,f2,f20,f21,f22,f23,f24,f25,f3,f4,f5,f6,f62,f7,f8,f9
0,2,10.07,0,9.89,002619,-,0,-,艾格拉斯,-,-,3.14,2,2.81,2.85,2.85,3.14,5792974165,4461350679,0,0.88,-3.09,-6.82,10.18,0.29,803959,2.46813e+08,5.88095e+07,11.58,5.66,16.12
1,2,1.97,0,153.74,002622,-,0,-,融钰集团,-,-,3.71,2,3.25,3.37,3.37,3.71,3116400000,3116316525,0,2.42,18.15,21.24,10.09,0.34,552777,1.9676e+08,4.25703e+07,13.65,6.58,-105.98
2,2,6.75,0,115.99,600778,-,1,-,友好集团,-,-,6.01,2,5.66,5.73,5.46,6.01,1872063026,1869631139,0,2.33,15.58,32.96,10.07,0.55,174752,1.02385e+08,-597763,6.41,5.62,24.19
3,2,5.07,0,-0.85,002247,-,0,-,聚力文化,-,-,3.17,2,2.84,2.88,2.88,3.17,2697258055,1726133095,0,1.3,3.93,-31.97,10.07,0.29,539597,1.66784e+08,3.12805e+07,11.46,9.91,30.93
4,2,9.75,0,445.73,600730,-,1,-,中国高科,-,-,5.91,2,5.46,5.5,5.37,5.91,3467136972,3467136972,0,1.79,9.04,22.11,10.06,0.54,422226,2.46376e+08,3.42279e+07,8.38,7.2,5891.34


In [142]:
#清理数据，读取成表格
share_tb = pd.DataFrame.from_dict(share_info)
share_tb = share_tb.loc[:,['f13', 'f12', 'f14', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f15', 'f16', 'f17', 'f18', 'f10', 'f8', 'f9', 'f23' ]]
share_tb.columns = ['交易所','序号','名称','最新价','涨跌幅','涨跌额','成交量(手)','成交额','振幅','最高','最低','今开','昨收','量比','换手率','市盈率(动态)','市净率']
share_tb['交易所'] = share_tb['交易所'].map(lambda x: np.where(x==1,'SH','SZ'))
share_tb

,交易所,序号,名称,最新价,涨跌幅,涨跌额,成交量(手),成交额,振幅,最高,最低,今开,昨收,量比,换手率,市盈率(动态),市净率
0,SZ,002619,艾格拉斯,3.14,10.18,0.29,803959,2.46813e+08,11.58,3.14,2.81,2.85,2.85,10.07,5.66,16.12,0.88
1,SZ,002622,融钰集团,3.71,10.09,0.34,552777,1.9676e+08,13.65,3.71,3.25,3.37,3.37,1.97,6.58,-105.98,2.42
2,SH,600778,友好集团,6.01,10.07,0.55,174752,1.02385e+08,6.41,6.01,5.66,5.73,5.46,6.75,5.62,24.19,2.33
3,SZ,002247,聚力文化,3.17,10.07,0.29,539597,1.66784e+08,11.46,3.17,2.84,2.88,2.88,5.07,9.91,30.93,1.3
4,SH,600730,中国高科,5.91,10.06,0.54,422226,2.46376e+08,8.38,5.91,5.46,5.5,5.37,9.75,7.2,5891.34,1.79
5,SZ,300152,科融环境,3.94,10.06,0.36,861562,3.369e+08,8.94,3.94,3.62,3.64,3.58,3.52,12.09,15.44,3.54
6,SZ,300311,任子行,8.54,10.05,0.78,914829,7.70324e+08,8.51,8.54,7.88,7.89,7.76,4.44,19.72,40.26,3.86
7,SZ,002492,恒基达鑫,5.04,10.04,0.46,179018,8.71065e+07,10.26,5.04,4.57,4.6,4.58,8,4.5,40.42,1.63
8,SH,603703,盛洋科技,16.23,10.03,1.48,60684,9.49649e+07,10.03,16.23,14.75,14.9,14.75,4.78,5.57,914.17,8.49
9,SZ,300202,聚龙股份,11.08,10.03,1.01,702348,7.62878e+08,6.95,11.08,10.38,10.5,10.07,1.44,16.54,145.76,3.76


In [145]:
#储存，encoding='utf_8_sig'防止中文乱码
today = '2019_10_12' 
share_tb.to_csv('share_shsz_'+today+'.csv', encoding='utf_8_sig', index=False)